```sql
-- COPYRIGHT (C) 2022 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.
```

# **INTRODUCTIONS**

---

The code is separated into multiple sections:

1. **[Flow Diagrams](#flow-diagrams)**
2. **[Dependencies](#dependencies)**
3. **[Input Layer](#input-layer)**
4. **[Static Layer](#static-layer)**


# **FLOW DIAGRAMS**

---

[![](https://mermaid.ink/img/pako:eNpdT19LAzEM_yolTzfYVkFEvAdhOh8GA8F7vO4hu2Z3xbu0tCkyxr67nVNQ85Tk9y85QectQQ2H0X90A0ZRavtmWJVKed9HDIPacMiitnikeEWaWLXPT81uphYLZWAQCanW2gdii4LLbp-WPOrXdRlWwWl_2eqb2_u7ByarDRTdo2raKgn2jvvZ7upb5P-iG0Fx3Z_sL-2mrRwPuHeCLEmFghrGnlQffQ6__GAOE8UJnS1Pni5rAzLQRAbq0lqM7wYMnwsvh3ImvVgnPkJ9wDHRHDCLb47cQS0x0w9p7bDcN32zzp_GyWqg)](https://mermaid-js.github.io/mermaid-live-editor/edit#pako:eNpdT19LAzEM_yolTzfYVkFEvAdhOh8GA8F7vO4hu2Z3xbu0tCkyxr67nVNQ85Tk9y85QectQQ2H0X90A0ZRavtmWJVKed9HDIPacMiitnikeEWaWLXPT81uphYLZWAQCanW2gdii4LLbp-WPOrXdRlWwWl_2eqb2_u7ByarDRTdo2raKgn2jvvZ7upb5P-iG0Fx3Z_sL-2mrRwPuHeCLEmFghrGnlQffQ6__GAOE8UJnS1Pni5rAzLQRAbq0lqM7wYMnwsvh3ImvVgnPkJ9wDHRHDCLb47cQS0x0w9p7bDcN32zzp_GyWqg)

Required steps:

1. **[CBS](TypedDataSet?$filter=((Geslacht%20eq%20%27T001038%27))%20and%20((RegioS%20eq%20%27NL01%20%20%27))%20and%20((Perioden%20eq%20%272020JJ00%27))%20and%20((BurgerlijkeStaat%20eq%20%27T001019%27)))**: contains the static `"inhabitants per age group"` data collected by CBS to be used for further processing.
2. **[Staging](#input-layer)**: inserts the raw `Source` data within the `[VWSSTAGE].[CBS_INHABITANTS_PER_AGE_GROUP]` table.
3. **[Inhabitants Per Municipality](#static-layer)**: inserts the `Staging` data and cast the correct `Database Types` within `[VWSSTATIC].[CBS_INHABITANTS_PER_AGE_GROUP]` using business logic implemented in `[dbo].[SP_INSERT_SL_CBS_INHABITANTS_PER_AGE_GROUP]`.

- **<font color=teal>SL</font>**: Static Layer


# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/DataFactory/Utils/Schemas.ipynb"
        // Additional dependencies (!NOTE! DO NOT FORGET THE COMMA (i.e. ,))
    ]
}
```

# **INPUT LAYER**

---

## **<span style='color:teal'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSSTAGE].[CBS_INHABITANTS_PER_AGE_GROUP]'))
CREATE TABLE [VWSSTAGE].[CBS_INHABITANTS_PER_AGE_GROUP] (
	[ID] [BIGINT] IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[SEX] [VARCHAR](100),
	[AGE] [VARCHAR](100),
	[MARITAL_STATUS] [VARCHAR](100),
	[REGIONS] [VARCHAR](100),
	[PERIODS] [VARCHAR](100),
    [POPULATION_ON_1ST_OF_JANUARY_1] [VARCHAR](100),
	[AVERAGE_POPULATION_2] [VARCHAR](100)
);
GO

# **STATIC LAYER**

---

## **<span style='color:teal'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSSTATIC].[CBS_INHABITANTS_PER_AGE_GROUP]'))
CREATE TABLE [VWSSTATIC].[CBS_INHABITANTS_PER_AGE_GROUP] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[SEX] [VARCHAR](100),
	[AGE] [VARCHAR](100),
	[MARITAL_STATUS] [VARCHAR](100),
	[REGIONS] [VARCHAR](100),
	[PERIODS] [VARCHAR](100),
    [POPULATION_ON_1ST_OF_JANUARY_1] [BIGINT],
	[AVERAGE_POPULATION_2] [DECIMAL](10,2)
);
GO

-- 2) CREATE INDEX(ES).....
DROP INDEX IF EXISTS [NCIX_DLI_CBS_INHABITANTS_PER_AGE_GROUP]
	ON [VWSSTATIC].[CBS_INHABITANTS_PER_AGE_GROUP]
GO

CREATE INDEX [NCIX_DLI_CBS_INHABITANTS_PER_AGE_GROUP]
	ON [VWSSTATIC].[CBS_INHABITANTS_PER_AGE_GROUP] (
		[DATE_LAST_INSERTED]
	);
GO

## **<span style='color:teal'>STORED PROCEDURES</span>**

In [ ]:
-- 1) CREATE STORED PROCEDURE(S): STAGING TO STATIC.....
CREATE OR ALTER PROCEDURE [dbo].[SP_INSERT_SL_CBS_INHABITANTS_PER_AGE_GROUP]
AS
BEGIN
    INSERT INTO [VWSSTATIC].[CBS_INHABITANTS_PER_AGE_GROUP] (
        [SEX],
        [AGE],
        [MARITAL_STATUS],
        [REGIONS],
        [PERIODS],
        [POPULATION_ON_1ST_OF_JANUARY_1],
        [AVERAGE_POPULATION_2]
    )
    SELECT
        [SEX],
        [AGE],
        [MARITAL_STATUS],
        [REGIONS],
        [PERIODS],
        [POPULATION_ON_1ST_OF_JANUARY_1],
        [AVERAGE_POPULATION_2]
    FROM
        [VWSSTAGE].[CBS_INHABITANTS_PER_AGE_GROUP]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[CBS_INHABITANTS_PER_AGE_GROUP])
END;
GO